ADD PROJECT 
- table where data is added : projects_wiroi_online

In [1]:
import requests
from bs4 import BeautifulSoup

login_url = "http://127.0.0.1:8000/admin/login/"
post_url = "http://127.0.0.1:8000/api/v1/projects/"

# Start a session
session = requests.Session()

# Get the login page to grab CSRF token
login_page = session.get(login_url)
soup = BeautifulSoup(login_page.text, "html.parser")
csrf_token = soup.find('input', attrs={'name': 'csrfmiddlewaretoken'}).get('value')

# Login payload
login_data = {
    "username": "adminuser",
    "password": "levon",
    "csrfmiddlewaretoken": csrf_token
}

# Django expects login via POST with CSRF and cookies
session.post(login_url, data=login_data, headers={"Referer": login_url})

# Now use the authenticated session to POST your project
project_data = {
    "name": "Test Project 8",
    "description": "A public mapping project for tracking deforestation patterns in South America.",
    "is_public": True,
    "is_active": True,
    "default_center_lat": -3.4653,
    "default_center_lng": -62.2159,
    "default_zoom_level": 6,
    "max_zoom": 18,
    "min_zoom": 3,
    "map_controls": {
        "zoomControl": True,
        "scaleControl": False,
        "fullScreenControl": True
    },
    "map_options": {
        "dragging": True,
        "scrollWheelZoom": False,
        "doubleClickZoom": True,
        "keyboard": True
    }
}

# You may still need to set the CSRF token manually in header
csrf_token = session.cookies.get('csrftoken')
headers = {
    "X-CSRFToken": csrf_token,
    "Referer": post_url
}

response = session.post(post_url, json=project_data, headers=headers)

print(response.status_code)
PROJECT_ID = response.json()['id']

201


In [2]:
PROJECT_ID

34

In [3]:
import requests
from bs4 import BeautifulSoup

# --- Config ---
BASE_URL = "http://127.0.0.1:8000"
LOGIN_URL = f"{BASE_URL}/admin/login/"
LAYER_GROUPS_URL = f"{BASE_URL}/api/v1/layer-groups/"

USERNAME = "adminuser"
PASSWORD = "levon"
# PROJECT_ID = 199  # Replace with the actual project ID

# --- Start session ---
session = requests.Session()

# --- Step 1: Login to Django admin ---

login_page = session.get(LOGIN_URL)
soup = BeautifulSoup(login_page.text, "html.parser")
csrf_token = soup.find("input", {"name": "csrfmiddlewaretoken"}).get("value")

login_data = {
    "username": USERNAME,
    "password": PASSWORD,
    "csrfmiddlewaretoken": csrf_token
}

session.post(LOGIN_URL, data=login_data, headers={"Referer": LOGIN_URL})

# --- Step 2: Create layer group ---
layer_group_payload = {
    "name": "Test Project 8",
    "project": PROJECT_ID,
    "display_order": 1,
    "is_visible_by_default": True,
    "is_expanded_by_default": False
}

csrf_token = session.cookies.get("csrftoken")
headers = {
    "X-CSRFToken": csrf_token,
    "Referer": LAYER_GROUPS_URL
}

response = session.post(LAYER_GROUPS_URL, json=layer_group_payload, headers=headers)


GROUP_ID = response.json()['id']
# --- Step 3: Print results ---
print("Status Code:", response.status_code)

try:
    print("Response:", response.json())
except Exception:
    print("Raw Response:", response.text)

Status Code: 201
Response: {'id': 50, 'name': 'Test Project 8', 'display_order': 1, 'is_visible_by_default': True, 'is_expanded_by_default': False, 'created_at': '2025-06-09T08:20:44.485318Z', 'updated_at': '2025-06-09T08:20:44.485365Z', 'project': 34}


In [29]:
import requests
from bs4 import BeautifulSoup

# Config
BASE_URL = "http://127.0.0.1:8000"
LOGIN_URL = f"{BASE_URL}/admin/login/"
CONSTRUCTOR_URL_TEMPLATE = f"{BASE_URL}/api/v1/constructor/{PROJECT_ID}/"

USERNAME = "adminuser"
PASSWORD = "levon"
PROJECT_ID = 1  # Replace with the actual project ID you want to load

# Start session
session = requests.Session()

# Step 1: Get CSRF token
login_page = session.get(LOGIN_URL)
soup = BeautifulSoup(login_page.text, "html.parser")
csrf_token = soup.find("input", {"name": "csrfmiddlewaretoken"}).get("value")

# Step 2: Log in
login_data = {
    "username": USERNAME,
    "password": PASSWORD,
    "csrfmiddlewaretoken": csrf_token
}
session.post(LOGIN_URL, data=login_data, headers={"Referer": LOGIN_URL})

# Step 3: Call constructor endpoint
constructor_url = CONSTRUCTOR_URL_TEMPLATE.format(project_id=PROJECT_ID)
response = session.get(constructor_url)

# Step 4: Output the result
print("Status Code:", response.status_code)
try:
    print("Response:", response.json())
except Exception:
    print("Raw Response:", response.text)

Status Code: 200
Response: {'project': {'id': 27, 'name': 'Test Project 8', 'description': 'A public mapping project for tracking deforestation patterns in South America.', 'default_center': {'lat': -3.4653, 'lng': -62.2159}, 'default_zoom': 6, 'min_zoom': 3, 'max_zoom': 18, 'map_controls': {'zoomControl': True, 'scaleControl': False, 'fullScreenControl': True}, 'map_options': {'dragging': True, 'keyboard': True, 'doubleClickZoom': True, 'scrollWheelZoom': False}}, 'basemaps': [], 'layer_groups': [], 'tools': []}


In [5]:
import requests
from bs4 import BeautifulSoup

session = requests.Session()

# --- Step 0: Login ---
login_url = "http://127.0.0.1:8000/admin/login/"
login_page = session.get(login_url)
soup = BeautifulSoup(login_page.text, "html.parser")
csrf_token = soup.find("input", attrs={"name": "csrfmiddlewaretoken"}).get("value")

login_data = {
    "username": "adminuser",
    "password": "levon",
    "csrfmiddlewaretoken": csrf_token
}
login_headers = {"Referer": login_url}
session.post(login_url, data=login_data, headers=login_headers)

# --- Step 1: Upload SHP file ---
upload_url = "http://127.0.0.1:8000/api/v1/upload/"
upload_headers = {
    "X-CSRFToken": session.cookies.get("csrftoken"),
    "Referer": upload_url
}

upload_response = session.post(
    upload_url,
    headers=upload_headers,
    files={"file": open("Test_Project/Clallam County-19 Eligible Locations.sqlite", "rb")})

if upload_response.status_code != 200:
    print("❌ Upload failed:", upload_response.text)
    exit()

upload_json = upload_response.json()
file_id = upload_json["file_id"]
print("FILE ID IS :",file_id)
file_type = upload_json["file_type"]
file_name = upload_json["file_name"]
source_crs = "ESRI:102008"

print("✅ File uploaded. File ID:", file_id)

# --- Step 2: Complete the upload ---
complete_upload_url = "http://127.0.0.1:8000/api/v1/complete_upload/"
complete_headers = {
    "X-CSRFToken": session.cookies.get("csrftoken"),
    "Referer": complete_upload_url,
    "Content-Type": "application/json"
}

payload = {
    "file_id": file_id,
    "file_type": file_type,
    "group_id": GROUP_ID,               # 👈 Use actual group ID
    "layer_name": "Location layer new",
    "layer_type_id": 5,          # 👈 Use actual layer type ID
    "source_crs": "EPSG:4326",
    "target_crs": "EPSG:4326",
    "description": "Imported via script",

    "is_visible": True,
    "is_public": True,
    "file_name": file_name,
"is_visible_by_defult":True
}

response = session.post(
    complete_upload_url,
    headers=complete_headers,
    json=payload
)

if response.status_code == 200:
    print("✅ Layer created!")
    print(response.json())
else:
    print("❌ Layer creation failed:")
    print(response.text)

FILE ID IS : f15deb04-4112-4637-adc5-2e5cb9dd5662
✅ File uploaded. File ID: f15deb04-4112-4637-adc5-2e5cb9dd5662
✅ Layer created!
{'success': True, 'layer_id': 136, 'layer_name': 'Location layer new', 'feature_count': 1167, 'message': 'Successfully imported 1167 features'}


In [6]:
layer_id = response.json()['layer_id']

In [7]:
import requests
from bs4 import BeautifulSoup

# --- Configuration ---
BASE_URL = "http://127.0.0.1:8000"
LOGIN_URL = f"{BASE_URL}/admin/login/"
APPLY_STYLE_URL = f"{BASE_URL}/api/v1/styles/STYLE_ID/apply_to_layer/"  # Replace STYLE_ID

USERNAME = "adminuser"
PASSWORD = "levon"
STYLE_ID = 4     # Replace with the style ID you want to apply
LAYER_ID = layer_id    # Replace with the target layer ID

# --- Step 1: Start session and get CSRF token ---
session = requests.Session()
login_page = session.get(LOGIN_URL)
soup = BeautifulSoup(login_page.text, "html.parser")
csrf_token = soup.find("input", {"name": "csrfmiddlewaretoken"}).get("value")

# --- Step 2: Log in ---
login_data = {
    "username": USERNAME,
    "password": PASSWORD,
    "csrfmiddlewaretoken": csrf_token
}
session.post(LOGIN_URL, data=login_data, headers={"Referer": LOGIN_URL})

# --- Step 3: Apply style to layer ---
csrf_token = session.cookies.get("csrftoken")
apply_url = APPLY_STYLE_URL.replace("STYLE_ID", str(STYLE_ID))

headers = {
    "X-CSRFToken": csrf_token,
    "Referer": apply_url
}

payload = {
    "layer_id": LAYER_ID
}

response = session.post(apply_url, json=payload, headers=headers)

# --- Step 4: Print result ---
print("Status Code:", response.status_code)
try:
    print("Response:", response.json())
except Exception:
    print("Raw Response:", response.text)

Status Code: 200
Response: {'message': 'Style applied successfully', 'layer_id': 136, 'layer_name': 'Location layer new'}


In [13]:
# Step 3: Create and Apply Custom Style
layer = {}
layer['style'] = {
  "color": "black",
  "radius": 10,
  "weight": 1,
  "fillColor": "#01fbff",
  "fillOpacity": 1
}
if "style" in layer:
    # Create the style first
    style_payload = {
        "name": f"Style for Location layer 2",
        "description": f"Auto-generated style for Location layer new",
        "style_type": "point",  # assuming vector; you can make this dynamic
        "style_definition": layer["style"],
        "is_system": False
    }

    style_create_resp = session.post(
        f"{BASE_URL}/api/v1/styles/",
        json=style_payload,
        headers={"X-CSRFToken": csrf_token, "Referer": BASE_URL}
    )
    print(style_create_resp.status_code)

    if style_create_resp.status_code == 201:
        style_id = style_create_resp.json()["id"]
        print("style_id",style_id)
        print(f"Custom style created with ID: {style_id}")

        # Apply the created style to the layer
        style_apply_url = f"{BASE_URL}/api/v1/styles/{style_id}/apply_to_layer/"
        apply_resp = session.post(
            style_apply_url,
            json={"layer_id": layer_id},
            headers={"X-CSRFToken": csrf_token, "Referer": BASE_URL}
        )

        if apply_resp.status_code == 200:
            print(f"Custom style applied to layer {layer_id}")
        else:
            print(f"Failed to apply style to layer {layer_id}")
    else:
        print(f"Failed to create style for layer {layer['layer_name']}")


KeyboardInterrupt: 

In [ ]:
# import requests
# from bs4 import BeautifulSoup
# import uuid

# # --- Setup ---
# BASE_URL = "http://127.0.0.1:8000"
# LOGIN_URL = f"{BASE_URL}/admin/login/"
# PROJECT_CREATE_URL = f"{BASE_URL}/api/v1/projects/"
# CLIENT_PROJECT_URL = f"{BASE_URL}/api/v1/client-projects/"

# USERNAME = "adminuser"
# PASSWORD = "levon"
# CLIENT_ID = 1  # Replace this with your actual client ID

# # Start session
# session = requests.Session()

# # --- Step 1: Get CSRF token from login page ---
# login_page = session.get(LOGIN_URL)
# soup = BeautifulSoup(login_page.text, "html.parser")
# csrf_token = soup.find('input', attrs={'name': 'csrfmiddlewaretoken'}).get('value')

# # --- Step 2: Log in ---
# login_data = {
#     "username": USERNAME,
#     "password": PASSWORD,
#     "csrfmiddlewaretoken": csrf_token
# }
# login_headers = {
#     "Referer": LOGIN_URL
# }
# session.post(LOGIN_URL, data=login_data, headers=login_headers)

# # --- Step 3: Create a new project ---
# project_data = {
#     "name": "rainforest_analysis_june_2025",
#     "description": "Monitoring deforestation in Amazon regions.",
#     "is_public": True,
#     "is_active": True,
#     "default_center_lat": -3.5,
#     "default_center_lng": -62.2,
#     "default_zoom_level": 6,
#     "min_zoom": 3,
#     "max_zoom": 18,
#     "map_controls": {
#         "zoomControl": True,
#         "scaleControl": False,
#         "fullScreenControl": True
#     },
#     "map_options": {
#         "dragging": True,
#         "scrollWheelZoom": True,
#         "doubleClickZoom": True,
#         "keyboard": True
#     }
# }

# csrf_token = session.cookies.get("csrftoken")
# headers = {
#     "X-CSRFToken": csrf_token,
#     "Referer": PROJECT_CREATE_URL
# }
# project_response = session.post(PROJECT_CREATE_URL, json=project_data, headers=headers)
# print("Project creation status:", project_response.status_code)
# project_json = project_response.json()
# print("Project response:", project_json)

# project_id = project_json.get("id")
# if not project_id:
#     raise Exception("Project creation failed or no project ID returned.")

# # --- Step 4: Assign the project to the client ---
# unique_link = f"client-{CLIENT_ID}-project-{project_id}-{uuid.uuid4().hex[:8]}"
# client_project_data = {
#     "client": CLIENT_ID,
#     "project": project_id,
#     "is_active": True,
#     "unique_link": unique_link
# }

# headers["Referer"] = CLIENT_PROJECT_URL
# client_project_response = session.post(CLIENT_PROJECT_URL, json=client_project_data, headers=headers)
# print("ClientProject creation status:", client_project_response.status_code)
# print("ClientProject response:", client_project_response.json())

Project creation status: 201
Project response: {'id': 16, 'name': 'rainforest_analysis_june_2025', 'description': 'Monitoring deforestation in Amazon regions.', 'is_public': True, 'is_active': True, 'default_center_lat': -3.5, 'default_center_lng': -62.2, 'default_zoom_level': 6, 'map_controls': {'zoomControl': True, 'scaleControl': False, 'fullScreenControl': True}, 'map_options': {'dragging': True, 'scrollWheelZoom': True, 'doubleClickZoom': True, 'keyboard': True}, 'max_zoom': 18, 'min_zoom': 3}
ClientProject creation status: 201
ClientProject response: {'id': 1, 'client': 1, 'project': 16, 'project_name': 'rainforest_analysis_june_2025', 'unique_link': 'client-1-project-16-d4a3b33b', 'is_active': True, 'created_at': '2025-06-07T15:11:14.853044Z', 'expires_at': None, 'last_accessed': None}


In [28]:
# login_data

adds a layer to your project not competlely though. layer info is being added to this table : project_layers_wiroi_online

In [ ]:

# import requests
# from bs4 import BeautifulSoup

# login_url = "http://127.0.0.1:8000/admin/login/"
# post_url = "http://127.0.0.1:8000/api/v1/upload/"


# session = requests.Session()

# # Get the login page to grab CSRF token
# login_page = session.get(login_url)
# soup = BeautifulSoup(login_page.text, "html.parser")
# csrf_token = soup.find('input', attrs={'name': 'csrfmiddlewaretoken'}).get('value')

# # Login payload
# login_data = {
#     "username": "adminuser",
#     "password": "levon",
#     "csrfmiddlewaretoken": csrf_token
# }

# # Django expects login via POST with CSRF and cookies
# session.post(login_url, data=login_data, headers={"Referer": login_url})

# # Now use the authenticated session to POST your project
# project_data = {
 
#   "project_layer_group": 16,
#   "layer_type": 5,
#   "name": "Location layer saturday",
#   "description": "loc",
#   "style":  {
#   "color": "#7A0000",
#   "weight": 1,
#   "fillColor": "#000000",
#   "fillOpacity": 0.6
# },
#   "z_index": 10,
#   "is_visible_by_default": True,
#   "min_zoom_visibility": 5,
#   "max_zoom_visibility": 18,
#   "marker_type": "null",
#   "marker_image_url": "null",
#   "marker_options": {},
#   "enable_clustering": False,
#   "clustering_options": {},
#   "enable_labels": False,
#   "label_options": {},
#   "data_source": "https://example.com/data/roads.geojson",
#   "attribution": "© OpenStreetMap Contributors"
# }

# # You may still need to set the CSRF token manually in header
# csrf_token = session.cookies.get('csrftoken')
# headers = {
#     "X-CSRFToken": csrf_token,
#     "Referer": post_url
# }

# response = session.post(post_url, json=project_data, headers=headers)

# print(response.status_code)
# print(response.text)

201
{"id":125,"project_layer_group":16,"project_name":"Missouri_test BEAD Analysis Dashboard","layer_type":5,"layer_type_name":"Point","name":"Location layer saturday","description":"loc","style":{"color":"#7A0000","weight":1,"fillColor":"#000000","fillOpacity":0.6},"z_index":10,"is_visible_by_default":true,"min_zoom_visibility":5,"max_zoom_visibility":18,"marker_type":"null","marker_image_url":"null","marker_options":{},"enable_clustering":false,"clustering_options":{},"enable_labels":false,"label_options":{},"feature_count":0,"data_source":"https://example.com/data/roads.geojson","attribution":"© OpenStreetMap Contributors","created_at":"2025-06-07T15:12:48.080777Z","updated_at":"2025-06-07T15:12:48.080787Z","last_data_update":null}


upload layer to a DB, shp file gave a problem, sqlite has too many features raised isseus as well

TypeError: request() got an unexpected keyword argument 'project_id'

In [21]:
import geopandas as gpd

loc = gpd.read_file("/Users/levon/Desktop/WebGIS/WebViewer-V2-BackEnd/aerial locations.sqlite")
loc[['geometry']].to_file("upload_locs.sqlite",driver='sqlite')

/opt/anaconda3/lib/python3.9/site-packages/pyogrio/raw.py:505: UserWarning: 'crs' was not provided.  The output dataset will not have projection information defined and may not be usable in other systems.
  warnings.warn(


In [5]:
complete_upload_url = "http://127.0.0.1:8000/api/v1/complete_upload/"
csrf_token = session.cookies.get("csrftoken")

headers = {
    "X-CSRFToken": csrf_token,
    "Referer": complete_upload_url,
    "Content-Type": "application/json"
}

payload = {
    "file_id": "1",
    "file_type": "shp",
    "group_id": 14,
    "layer_name": "Location layer",
    "layer_type_id": 5,
    "source_crs": "EPSG:4326",
    "target_crs": "EPSG:4326",
    "description": "Test import from shapefile",
    "is_visible": True,
    "is_public": False
}

response = session.post(complete_upload_url, json=payload, headers=headers)

if response.status_code == 200:
    print("✅ Upload completed and layer created:")
    print(response.json())
else:
    print(f"❌ Error: {response.status_code}")
    print(response.text)

❌ Error: 404
{"error":"File not found. It may have expired."}


In [14]:
import geopandas as gpd

/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [15]:
d = gpd.read_file("/Users/levon/Desktop/WebGIS/WebViewer-V2-BackEnd/Test_Project/Clallam County-19 Eligible Locations.sqlite")

In [18]:
d['x'] = d.geometry.x
d['y'] = d.geometry.y

In [20]:
from fiona.drvsupport import supported_drivers

supported_drivers['sqlite'] = 'rw'
d[['geometry']].to_file("/Users/levon/Desktop/WebGIS/WebViewer-V2-BackEnd/Test_Project/Clallam County-19 Eligible Locations 2.sqlite",driver='sqlite')

/opt/anaconda3/lib/python3.9/site-packages/pyogrio/raw.py:505: UserWarning: 'crs' was not provided.  The output dataset will not have projection information defined and may not be usable in other systems.
  warnings.warn(


ADD POPUP TEMPLATE

In [28]:
import requests
from bs4 import BeautifulSoup

# Base configuration
BASE_URL = "http://127.0.0.1:8000"
LOGIN_URL = f"{BASE_URL}/admin/login/"
POPUP_TEMPLATE_URL = f"{BASE_URL}/api/v1/popup-templates/"

USERNAME = "adminuser"
PASSWORD = "levon"

# Start a session
session = requests.Session()

# Step 1: Get CSRF token from login page
login_page = session.get(LOGIN_URL)
soup = BeautifulSoup(login_page.text, "html.parser")
csrf_token = soup.find("input", attrs={"name": "csrfmiddlewaretoken"}).get("value")

# Step 2: Perform login
login_data = {
    "username": USERNAME,
    "password": PASSWORD,
    "csrfmiddlewaretoken": csrf_token
}

headers = {
    "Referer": LOGIN_URL
}

login_response = session.post(LOGIN_URL, data=login_data, headers=headers)

print(login_response)
# Step 3: Create popup template
template_data = {
    "name": "Basic Info Popup",
    "description": "Shows basic fields in a popup",
    "html_template": "<strong>{{ name }}</strong><br>{{ description }}<br>Value: {{ value }}",
    "field_mappings": "{\"name\": \"Name\", \"description\": \"Description\", \"value\": \"Value\"}",
    "css_styles": ".popup { font-size: 14px; color: #333; }",
    "is_system": False
}

headers = {
    "X-CSRFToken": session.cookies.get("csrftoken"),
    "Referer": POPUP_TEMPLATE_URL
}

response = session.post(
    POPUP_TEMPLATE_URL,
    json=template_data,
    headers=headers
)

if response.status_code == 201:
    template_id = response.json().get("id")
    print(f"✅ Created popup template with ID: {template_id}")
else:
    print(f"❌ Failed to create template: {response.status_code}")
    print(response.text)

<Response [404]>
✅ Created popup template with ID: 1


In [ ]:
# Replace with your actual layer ID and the created template ID
layer_id = 123
template_id = template_id  # already defined from previous step

LAYER_UPDATE_URL = f"{BASE_URL}/api/v1/project-layers/{layer_id}/"

# Prepare the payload and headers
update_payload = {
    "popup_template_id": template_id
}

headers = {
    "X-CSRFToken": session.cookies.get("csrftoken"),
    "Referer": LAYER_UPDATE_URL
}

update_response = session.patch(
    LAYER_UPDATE_URL,
    json=update_payload,
    headers=headers
)

if update_response.status_code in [200, 202]:
    print(f"✅ Layer {layer_id} successfully updated with template {template_id}")
else:
    print(f"❌ Failed to update layer: {update_response.status_code}")
    print(update_response.text)

❌ Failed to update layer: 404
<!DOCTYPE html>
<html lang="en">
<head>
  <meta http-equiv="content-type" content="text/html; charset=utf-8">
  <title>Page not found at /api/v1/project-layers/123/</title>
  <meta name="robots" content="NONE,NOARCHIVE">
  <style type="text/css">
    html * { padding:0; margin:0; }
    body * { padding:10px 20px; }
    body * * { padding:0; }
    body { font:small sans-serif; background:#eee; color:#000; }
    body>div { border-bottom:1px solid #ddd; }
    h1 { font-weight:normal; margin-bottom:.4em; }
    h1 span { font-size:60%; color:#666; font-weight:normal; }
    table { border:none; border-collapse: collapse; width:100%; }
    td, th { vertical-align:top; padding:2px 3px; }
    th { width:12em; text-align:right; color:#666; padding-right:.5em; }
    #info { background:#f6f6f6; }
    #info ol { margin: 0.5em 4em; }
    #info ol li { font-family: monospace; }
    #summary { background: #ffc; }
    #explanation { background:#eee; border-bottom: 0px none